In [1]:
import cv2
import numpy as np

# 'path to input image/video'

# 'path to yolo config file' 
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.cfg
CONFIG='./yolo/yolov3.cfg'

# 'path to text file containing class names'
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.txt
CLASSES='./yolo/yolov3.txt'

# 'path to yolo pre-trained weights' 
# wget https://pjreddie.com/media/files/yolov3.weights
WEIGHTS='./yolo/yolov3.weights'

In [2]:
import os  
print(os.path.exists(CLASSES))
print(os.path.exists(CONFIG))
print(os.path.exists(WEIGHTS))

True
True
True


In [3]:
classes = None
with open(CLASSES, 'r') as f:
     classes = [line.strip() for line in f.readlines()]
        
scale = 0.00392
conf_threshold = 0.5
nms_threshold = 0.4

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [4]:
def get_output_layers(net): 
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [5]:
def processImage(image, index):

    Width = image.shape[1]
    Height = image.shape[0]

    # read pre-trained model and config file
    net = cv2.dnn.readNet(WEIGHTS, CONFIG)

    # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                    
    # apply non-max suppression
    
    classified_objects = []
    for i in class_ids:
        classified_objects.append(classes[i])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
    
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    # display output image    
    
    #cv2.imshow(out_image_name, image)
    # wait until any key is pressed
    #cv2.waitKey()
    # save output image to disk
    return classified_objects

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = 'imjennim'
pr_classes = {}
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        try: 
            print(directory + '/' + filename)
            temp = mpimg.imread(directory + '/' + filename)
            pr_classes[filename] = processImage(temp, filename)
        except:
            continue

imjennim/12394053_458510841000340_1115240788_n.jpg
imjennim/10731935_1520091278281341_162288298_n.jpg
imjennim/10852689_312466782275263_1188918758_n.jpg
imjennim/20225530_659931897538716_6981711296630620160_n.jpg
imjennim/16790289_1005605026239586_2618640718557609984_n.jpg
imjennim/924274_1400198843567492_953814237_n.jpg
imjennim/914541_178001869067937_317952802_n.jpg
imjennim/10665397_1536240213274395_672510900_n.jpg
imjennim/18252217_330645714065715_6423891445421178880_n.jpg
imjennim/12543191_489113014621226_1028111142_n.jpg
imjennim/10632509_728866430518977_571519300_n.jpg
imjennim/10903518_1552829021670066_1126830646_n.jpg
imjennim/53036392_1199543756870430_4123373395144877902_n.jpg
imjennim/22344297_840468949465159_1810407725482377216_n.jpg
imjennim/10499272_239379112939395_1562831045_n.jpg
imjennim/11380925_1438669889773402_58711808_n.jpg
imjennim/916335_217715501743277_1212914376_n.jpg
imjennim/914327_1431637340419663_132040812_n.jpg
imjennim/11849323_1051525561525776_1078539727

imjennim/11378724_1453520188296228_623457802_n.jpg
imjennim/14574169_1703327946624811_8408890733947256832_n.jpg
imjennim/11235962_1063303703698615_814732799_n.jpg
imjennim/20987570_340902389682808_5844333437581787136_n.jpg
imjennim/10817826_1510279065915847_1452544702_n.jpg
imjennim/39059703_993680744168856_8742433255589937152_n.jpg
imjennim/18514024_244553799358193_6617714188389187584_n.jpg
imjennim/29088440_130922054407728_4050173942747365376_n.jpg
imjennim/11311637_849319518468777_1594978282_n.jpg
imjennim/916980_624369337626897_1207002550_n.jpg
imjennim/1171229_487013371396590_149134770_n.jpg
imjennim/11203352_1581909165410241_635207692_n.jpg
imjennim/12750049_520861348122555_1318754847_n.jpg
imjennim/12230781_1234920946600088_96272200_n.jpg
imjennim/39878241_963991590453627_4186241397425121062_n.jpg
imjennim/53093020_123713512066533_6053880981485212274_n.jpg
imjennim/10683816_722521391146710_1692437987_n.jpg
imjennim/41260478_311507126333261_1685341449070993167_n.jpg
imjennim/1191

imjennim/11283276_1380407858933022_59804482_n.jpg
imjennim/11186992_477838939029936_1601290030_n.jpg
imjennim/21224167_354144895005492_7532628770323169280_n.jpg
imjennim/30079429_795529057311472_5123873371125186560_n.jpg
imjennim/15275555_229370817468566_1465656516018176000_n.jpg
imjennim/36525049_254000775407065_1538985235692126208_n.jpg
imjennim/11335494_974146609284591_58017295_n.jpg
imjennim/10471810_701150986622935_290884766_n.jpg
imjennim/10533418_569021563230909_408599881_n.jpg
imjennim/15876721_1778736529118852_7941060540762161152_n.jpg
imjennim/53659146_2090542951000402_578658074288739160_n.jpg
imjennim/39605492_2174153839466975_7477643173070635008_n.jpg
imjennim/30829642_378166102685087_8678774027717705728_n.jpg
imjennim/12724927_2005112063047187_391598942_n.jpg
imjennim/42128589_1957959374317878_1551454356743492344_n.jpg
imjennim/11373681_367229173476640_1212676570_n.jpg
imjennim/10725152_1516906178547695_766092408_n.jpg
imjennim/11208022_102458996754584_519047804_n.jpg
imje

imjennim/1742336_1402961853295000_1116498211_n.jpg
imjennim/11382940_1604796249792890_254969650_n.jpg
imjennim/11351678_384871575029643_1345107797_n.jpg
imjennim/54512508_697126687352202_3422361105333043800_n.jpg
imjennim/11240417_1618884058354515_803828981_n.jpg
imjennim/11208064_1574624362818898_246955425_n.jpg
imjennim/32063854_2132819813665497_8057918732506234880_n.jpg
imjennim/11187064_1589153338021863_1371648683_n.jpg
imjennim/11201519_975101755844644_1342875335_n.jpg
imjennim/11282004_639050506196793_1846342471_n.jpg
imjennim/25021551_1790898694537312_2092784314612711424_n.jpg
imjennim/12716943_233689660298175_192874277_n.jpg
imjennim/43031549_371339526940178_1240825273315896464_n.jpg
imjennim/14276419_522782157920551_1093197944_n.jpg
imjennim/11192647_1598996760387150_1717005518_n.jpg
imjennim/38996672_281653615772237_5112307200290717696_n.jpg
imjennim/14448304_305628516473128_9072198344905850880_n.jpg
imjennim/32903597_184310765621838_5027675191845584896_n.jpg
imjennim/2973818

imjennim/10472038_1424016907878301_1249715390_n.jpg
imjennim/12530993_965212150192502_725947404_n.jpg
imjennim/13181406_524825621059573_1995670533_n.jpg
imjennim/10986364_1602281149983994_515546714_n.jpg
imjennim/13651983_991649807618192_1072185162_n.jpg
imjennim/21819993_117549335588409_2071415701263876096_n.jpg
imjennim/53600445_2112273485554807_4791689954651728654_n.jpg
imjennim/40264991_435844763608634_453948115434940003_n.jpg
imjennim/923582_1439133123001398_184996339_n.jpg
imjennim/12912739_1680571582193611_2030398574_n.jpg
imjennim/26383145_144260139577325_8995024620184666112_n.jpg
imjennim/11254078_460236114142536_1596381976_n.jpg
imjennim/11809969_507446506085503_1519012875_n.jpg
imjennim/10454082_491197517679291_889969236_n.jpg
imjennim/10954861_444472292370234_2031550353_n.jpg
imjennim/11247933_1599228430358608_1263916604_n.jpg
imjennim/12965818_1758201301081846_991200495_n.jpg
imjennim/15625206_342394239474993_187032751860350976_n.jpg
imjennim/14280524_976301462492878_47685

In [8]:
import csv
w = csv.writer(open("imjennim_output.csv", "w"))
for key, val in pr_classes.items():
    w.writerow([key, val])